__Simulación de dinámica molecular: Etanol en agua__\
Autor: Jesus Alvarado\
2022

Iniciamos el cuaderno del colab, activando la GPU \
Conectamos con Google Drive

In [ ]:
%%bash
lscpu
nvidia-smi

__1. Instalación de Gromacs__

Descargamos y descomprimimos cmake

In [ ]:
%%bash
wget https://github.com/Kitware/CMake/releases/download/v3.20.0-rc1/cmake-3.20.0-rc1.tar.gz
tar xfz cmake-3.20.0-rc1.tar.gz

La instalación de cmake (siguiente celda) demora unos 5 minutos

In [ ]:
%%bash
mkdir /content/cmake-3.20.0-rc1/build
cd /content/cmake-3.20.0-rc1/build
cmake /content/cmake-3.20.0-rc1/
make -j 2
make install

Descarga, descompresión e instalación de gromacs (demoró aprox 30 minutos)

In [ ]:
%%bash
wget ftp://ftp.gromacs.org/gromacs/gromacs-2021.tar.gz
tar xfz gromacs-2021.tar.gz
cd /content/gromacs-2021
mkdir /content/gromacs-2021/build
cd /content/gromacs-2021/build
cmake .. -DGMX_BUILD_OWN_FFTW=ON -DREGRESSIONTEST_DOWNLOAD=ON -DGMX_GPU=CUDA
make -j 2
make install
source /usr/local/gromacs/bin/GMXRC

In [ ]:
%%bash
source /usr/local/gromacs/bin/GMXRC
mkdir /content/grojobs
cd /content/grojobs
gmx

In [50]:
!export GMXLIB=/usr/local/gromacs/share/gromacs/top

__2. Insertamos una molécula de etanol__ \
La molécula fue creada con smiles usando la web cheminfo \
Se empleó chimera para guardarla en formato PDB

Creación de gro, itp y top

In [51]:
#!ls /usr/share/gromacs/top
!ls /usr/local/gromacs/share/gromacs/top

amber03.ff	   electroneg.dat	 gromos53a6.ff	   spc.itp
amber94.ff	   elements.dat		 gromos54a7.ff	   specbond.dat
amber96.ff	   export.dlg		 ha-shift.dat	   ss.map
amber99.ff	   ffG43a1.itp		 ions.itp	   surface.dat
amber99sb.ff	   ffG43a2.itp		 nsfactor.dat	   sw.itp
amber99sb-ildn.ff  ffG45a3.itp		 oplsaa.ff	   table6-10.xvg
amberGS.ff	   ffG53a5.itp		 phbres.dat	   table6-11.xvg
atommass.dat	   ffG53a6.itp		 ps.m2p		   table6-12.xvg
atom_nom.tbl	   ffoplsaa.itp		 random.dat	   table6-8.xvg
bonds.dlg	   ffoplsaa-n.tst	 README		   table6-9.xvg
ca-shift.dat	   flexspce.itp		 refi_aa.dat	   tip3p.itp
cb-shift.dat	   flexspc.itp		 residues.dtd	   tip4p.gro
charmm27.ff	   flexwat-ferguson.itp  residues.xml	   tip4p.itp
co-shift.dat	   gromos43a1.ff	 residuetypes.dat  tip5p.gro
defselection.dat   gromos43a2.ff	 sfactor.dat	   vdw-msms.dat
dgsolv.dat	   gromos45a3.ff	 spc216.gro	   vdwradii.dat
edissoc.dat	   gromos53a5.ff	 spce.itp	   xlateat.dat


__3. Force Field__

In [64]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx pdb2gmx -f etoh_chimera.pdb -o etoh.gro -water tip5p -ff oplsaa

Using the Oplsaa force field in directory oplsaa.ff

going to rename oplsaa.ff/aminoacids.r2b
Reading etoh_chimera.pdb...
Read 'EtOH', 9 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 1 residues with 9 atoms

  chain  #res #atoms

  1 ' '     1      9  

All occupancies are one

Reading residue database... (Oplsaa)

Processing chain 1 (9 atoms, 1 residues)

Problem with chain definition, or missing terminal residues. This chain does not appear to contain a recognized chain molecule. If this is incorrect, you can edit residuetypes.dat to modify the behavior.

Checking for duplicate atoms....

Generating any missing hydrogen atoms and/or adding termini.

Now there are 1 residues with 9 atoms

Making bonds...

Number of bonds was 8, now 8

Generating angles, dihedrals and pairs...

Making cmap torsions...

There are   12 dihedrals,    0 impropers,   13 angles
            12 pairs,        8 bonds 

                      :-) GROMACS - gmx pdb2gmx, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

__4. Caja de agua__

Creación de una caja cúbica con 1 nm de distancia hacia el borde

In [65]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx editconf -f etoh.gro -o newbox.gro -d 1.0 -bt cubic

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 9 atoms
Volume: 0.0140061 nm^3, corresponds to roughly 0 electrons
No velocities found
    system size :  0.239  0.204  0.289 (nm)
    diameter    :  0.356               (nm)
    center      :  0.195 -0.004 -0.027 (nm)
    box vectors :  0.239  0.203  0.289 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :   0.01               (nm^3)
    shift       :  0.983  1.182  1.205 (nm)
new center      :  1.178  1.178  1.178 (nm)
new box vectors :  2.356  2.356  2.356 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :  13.08               (nm^3)


                      :-) GROMACS - gmx editconf, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Se usan los modelos de solvente que están por defecto en la carpeta:
/usr/gromacs/share/gromacs/top

In [66]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx solvate -cp newbox.gro -cs tip5p.gro -o solv.gro -p topol.top


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for 418 solvent molecules with resname (SOL) to topology file (topol.top)


                      :-) GROMACS - gmx solvate, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

__5. Iones__

In [70]:
with open("ions.mdp","w") as f:
  f.write("; ions.mdp - used as input into grompp to generate ions.tpr")
  f.write("; Parameters describing what to do, when to stop and what to save")
  f.write("integrator  = steep         ; Algorithm (steep = steepest descent minimization)")
  f.write("emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm")
  f.write("emstep      = 0.01          ; Minimization step size")
  f.write("nsteps      = 50000         ; Maximum number of (minimization) steps to perform")
  f.write("; Parameters describing how to find the neighbors of each atom and how to calculate the interactions")
  f.write("nstlist         = 1         ; Frequency to update the neighbor list and long range forces")
  f.write("cutoff-scheme	= Verlet    ; Buffered neighbor searching") 
  f.write("ns_type         = grid      ; Method to determine neighbor list (simple, grid)")
  f.write("coulombtype     = cutoff    ; Treatment of long range electrostatic interactions")
  f.write("rcoulomb        = 1.0       ; Short-range electrostatic cut-off")
  f.write("rvdw            = 1.0       ; Short-range Van der Waals cut-off")
  f.write("pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions")

In [71]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f ions.mdp -c solv.gro -p topol.top -o ions.tpr

Setting the LD random seed to -1153441894

Generated 330891 of the 330891 non-bonded parameter combinations

Generated 330891 of the 330891 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Other'

Excluding 2 bonded neighbours molecule type 'SOL'

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are:     1      Other residues
There are:   418      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 0 Mb of data


                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [72]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx genion -s ions.tpr -o solv_ions.gro -p topol.top -pname NA -nname CL -neutral

                       :-) GROMACS - gmx genion, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

__6. MINIMIZAR__

In [74]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f minim.mdp -c solv_ions.gro -p topol.top -o em.tpr

Setting the LD random seed to -151520001

Generated 330891 of the 330891 non-bonded parameter combinations

Generated 330891 of the 330891 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Other'

Excluding 2 bonded neighbours molecule type 'SOL'

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are:     1      Other residues
There are:   418      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.118 0.118 0.118

Estimate for the relative computational load of the PME mesh part: 0.21

This run will generate roughly 0 Mb of data


                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [75]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx mdrun -v -deffnm em -pin on -nb gpu

                       :-) GROMACS - gmx mdrun, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murt

-----GRAFICANDO RESULTADOS MINIM------

In [77]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "9 0" | gmx energy -f em.edr -o potential.xvg


Statistics over 100 steps [ 0.0000 through 99.0000 ps ], 1 data sets
All statistics are over 79 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                  -15769.2        740    1999.81   -4880.97  (kJ/mol)


                       :-) GROMACS - gmx energy, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

__7. EQUILIBRIO 1 (NVT)__

Necesitamos 3 cosas:\
(1) Primero requerimos crear un archivo de índice: index.ndx
gmx make_ndx -f em.gro -o index.ndx
(puedes generarlo localmente)
Se selecionaron: "2", luego "4" y luego "q" para guardar y salir \
(2) Se descargó el archivo nvt.mdp \
(3) En la fila tc-grps se editó, colocando los archivos que llamaremos:\
tc-grps                 = Other SOL ;Protein Non-Protein   ; two coupling groups - more accurate

In [82]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -n index.ndx -o nvt.tpr

Setting the LD random seed to -68259525

Generated 330891 of the 330891 non-bonded parameter combinations

Generated 330891 of the 330891 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Other'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Setting gen_seed to -839401634

Velocities were taken from a Maxwell distribution at 300 K

Cleaning up constraints and constant bonded interactions with virtual sites

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.034 nm, buffer size 0.034 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 16x16x16, spacing 0.147 0.147 0.147

Estimate for the relative computational load of the PME mesh part: 0.19

This run wi

                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [85]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx mdrun -v -deffnm nvt -nb gpu

                       :-) GROMACS - gmx mdrun, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murt

-----GRAFICANDO RESULTADOS NVT------

In [87]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "15 0" | gmx energy -f nvt.edr -o temperature.xvg


Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 300.823       0.42    8.79918   -1.11226  (K)


                       :-) GROMACS - gmx energy, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

__7. EQUILIBRIO 2 (NPT)__

In [88]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -n index.ndx -o npt.tpr

Setting the LD random seed to -1615352065

Generated 330891 of the 330891 non-bonded parameter combinations

Generated 330891 of the 330891 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Other'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites

The center of mass of the position restraint coord's is  1.267  1.270  1.165

The center of mass of the position restraint coord's is  1.267  1.270  1.165

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.034 nm, buffer size 0.034 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole trajectory

Using frame at t = 100 ps

Sta

                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [89]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx mdrun -v -deffnm npt -pin on -nb gpu

                       :-) GROMACS - gmx mdrun, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murt

----GRAFICANDO RESULTADOS NPT------

In [91]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "17 0" | gmx energy -f npt.edr -o pressure.xvg


Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                     33.126         19      621.5    39.7205  (bar)


                       :-) GROMACS - gmx energy, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

In [92]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "23 0" | gmx energy -f npt.edr -o density.xvg


Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Density                     987.141          1    13.1763    2.87166  (kg/m^3)


                       :-) GROMACS - gmx energy, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

__8. PRODUCCIÓN__

Descargamos el archivo md.mdp y editamos de forma análoga en tc-grps:\
tc-grps                 = Other SOL ;Protein Non-Protein   ; two coupling groups - more accurate

In [93]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr

Setting the LD random seed to -824328665

Generated 330891 of the 330891 non-bonded parameter combinations

Generated 330891 of the 330891 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Other'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are:     1      Other residues
There are:   418      Water residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.034 nm, buffer size 0.034 nm

Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole trajectory



                       :-) GROMACS - gmx grompp, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Demoraría unas 3 horas con 8 CPU's vs 2 minutos con GPU

In [94]:
%%bash
source /usr/local/gromacs/bin/GMXRC
gmx mdrun -v -deffnm md_0_1 -pin on -nb gpu

                       :-) GROMACS - gmx mdrun, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murt

__9. ANÁLISIS__\
2 (EtOH)\
0 (System)

In [95]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "2 0" | gmx trjconv -s md_0_1.tpr -f md_0_1.xtc -o md_0_1_noPBC.xtc -pbc mol -center

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 2: 'EtOH'
Select group for output
Selected 0: 'System'


                      :-) GROMACS - gmx trjconv, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

----RMSD----\
"-tu ns" es para asegurarse de que los outputs están en nanosegundos\
Se creará: rmsd.xvg

In [96]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "2 0" | gmx rms -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsd.xvg -tu ns

Selected 2: 'EtOH'
Selected 0: 'System'


                        :-) GROMACS - gmx rms, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murto

\
Se creará: rmsd_xtal.xvg

In [97]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "2 0" | gmx rms -s em.tpr -f md_0_1_noPBC.xtc -o rmsd_xtal.xvg -tu ns

Selected 2: 'EtOH'
Selected 0: 'System'


                        :-) GROMACS - gmx rms, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Murto

In [98]:
%%bash
source /usr/local/gromacs/bin/GMXRC
printf "2" | gmx gyrate -s md_0_1.tpr -f md_0_1_noPBC.xtc -o gyrate.xvg

Selected 2: 'EtOH'


                       :-) GROMACS - gmx gyrate, 2021 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Movemos las gráficas de los resultados a una carpeta que creamos (mds_etanol)

In [103]:
!mkdir mds_etanol
!cp *.xvg /content/mds_etanol/.
!ls /mds_etanol

ls: cannot access '/mds_etanol': No such file or directory


Comprimimos la carpeta y la descargamos

In [110]:
!ls /content/mds_etanol/
!zip -r /content/mds_etanol.zip /content/mds_etanol
from google.colab import files
files.download('/content/mds_etanol.zip')

density.xvg  potential.xvg  rmsd_xtal.xvg  temperature.xvg
gyrate.xvg   pressure.xvg   rmsd.xvg
  adding: content/mds_etanol/ (stored 0%)
  adding: content/mds_etanol/pressure.xvg (deflated 59%)
  adding: content/mds_etanol/rmsd_xtal.xvg (deflated 65%)
  adding: content/mds_etanol/rmsd.xvg (deflated 65%)
  adding: content/mds_etanol/potential.xvg (deflated 61%)
  adding: content/mds_etanol/gyrate.xvg (deflated 66%)
  adding: content/mds_etanol/temperature.xvg (deflated 62%)
  adding: content/mds_etanol/density.xvg (deflated 61%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>